# 3. **F1 Win Prediction Project - Data Preprocessing**

# Table of Contents
1. [Introduction](#introduction)
2. [Data Types and Integrity Checks](#data-types-and-integrity-checks)
    - [Data Types](#data-types)
    - [Missing Data](#missing-data)
    - [Duplicate Rows](#duplicate-rows)
    - [Uniqueness of Data](#uniqueness-of-data)
    - [Data Range](#data-range)
    - [Consistency](#consistency)
    - [Correctness](#correctness)
    - [Data Integrity](#data-integrity)
3. [Data Preprocessing](#data-preprocessing)
    - [Importing Libraries and Notebook Setup](#importing-libraries-and-notebook-setup)
    - [Initial DataFrame Information](#initial-dataframe-information)
    - [Creating New Index Column](#creating-new-index-column)
    - [Rename Columns](#rename-columns)
    - [Drop Redundant Columns](#drop-redundant-columns)
    - [Changing Data Types](#changing-data-types)
    - [Dropping Duplicates](#dropping-duplicates)
    - [Handling Unreasonable Data Ranges](#handling-unreasonable-data-ranges)
4. [Feature Engineering / Transformation](#feature-engineering--transformation)
    - [Constructor Points at Stage of Season](#constructor-points-at-stage-of-season)
    - [Driver Points at Stage of Season](#driver-points-at-stage-of-season)
    - [Other Feature Engineering Ideas](#other-feature-engineering-ideas)
5. [One Hot Encoding](#one-hot-encoding)
6. [Reviewing 'fastest_lap_from_last_race' Column](#reviewing-fastest_lap_from_last_race-column)
7. [Exporting the Processed DataFrame](#exporting-the-processed-dataframe)


### **Areas to Fix**

- **Data Types**: Ensure that each column has the appropriate data type for the kind of data it contains. For instance, categorical data should not be typed as numeric and vice versa. If any columns are meant to be categorical or date/time but are currently recognized as 'object' or 'int64', they should be converted to the proper data type.

- **Missing Data**: The dataset contains columns with high percentages of missing values, such as 'laps_in_previous_race' and 'laps_completed_in_previous_races'. I will need to decide how to handle these, whether by imputation, deletion, or acquisition of more data if possible. For columns with a small amount of missing data, imputation might be feasible, while for those with a large percentage, it might be more appropriate to consider dropping the column.

- **Duplicate Rows**: I will also check for any duplicate rows that might skew the analysis. If duplicates are not meaningful for the study, they will be removed.

- **Data Range**: Next, I will verify the range of values in numerical columns. For instance, if 'year' has a minimum value that's in the future or a past date that's not plausible (e.g. outside the date ranges I collected), these could be data entry/collection errors. 

- **Consistency**: Ensure that the data is consistent throughout the dataset. For example, if 'country' and 'nationality_of_circuit' are very similar and basically  represent the same information, so they should possibly merged (or dropped).

- **Correctness**: For columns with 'inf' values for uniqueness, ensure they are correctly calculated. An 'inf' value might indicate a division by zero error, suggesting that the column might be entirely unique or entirely composed of a single value, each of which has different implications.

- **Normalization/Standardization**: For machine learning purposes, I may need to standardize or normalize numerical data to ensure that the scale of the data does not unduly influence the model. This may be done at the modelling stage, rather than here, but it's a consideration I need to think about.

### Importing Libraries and Notebook Setup

In [22]:
# Install libraries
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

In [23]:
process_df = pd.read_csv('C:/Users/Alex/OneDrive/BrainStation/Data_Science_Bootcamp/Capstone_Project/capstone-Aboard89/data/data_analysis.csv')

In [24]:
process_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11235 entries, 0 to 11234
Data columns (total 36 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Index                             11235 non-null  int64  
 1   resultId                          11235 non-null  int64  
 2   raceId                            11235 non-null  int64  
 3   year                              11235 non-null  int64  
 4   race                              11235 non-null  object 
 5   country                           11235 non-null  object 
 6   nationality_of_circuit            11235 non-null  object 
 7   driverId                          11235 non-null  int64  
 8   number                            11235 non-null  int64  
 9   driver_name                       11235 non-null  object 
 10  F2_champion                       11235 non-null  int64  
 11  Former_F1_World_Champion          11235 non-null  int64  
 12  Nati

In [25]:
pd.set_option('display.max_columns', None)
process_df.head()

,Index,resultId,raceId,year,race,country,nationality_of_circuit,driverId,number,driver_name,F2_champion,Former_F1_World_Champion,Nationality,home_race,constructorId,constructor,engine_manufacturer,constructor_nationality,number.1,starting_grid_position,positionOrder,points,points_in_previous_race,laps,laps_completed_in_previous_races,time,milliseconds,fastestLap_ms,fastest_lap_from_last_race,number_of_pit_stops,average_time_lost_in_pits,statusId,status,major_regulation_change,laps_in_previous_race,race_win
0,1,4721,240,1995,Brazilian Grand Prix,brazil,Brazilian,30,1,Michael Schumacher,0,1,German,0,22,Benetton,Renault,Italian,1,2,1,10.0,NaN,71,NaN,01:38:34.154000,5914154,81009,NaN,3,31.83,1,Finished,0,NaN,1
1,1,4724,240,1995,Brazilian Grand Prix,brazil,Brazilian,57,8,Mika Häkkinen,0,0,Finnish,0,1,McLaren,Mercedes,British,8,7,4,3.0,NaN,70,NaN,\N,\N,Not Found,NaN,0,0.00,11,+1 Lap,0,NaN,0
2,1,4746,240,1995,Brazilian Grand Prix,brazil,Brazilian,94,23,Pierluigi Martini,0,0,Italian,0,18,Minardi,Ford,Italian,23,17,26,0.0,NaN,0,NaN,\N,\N,Not Found,NaN,0,0.00,6,Gearbox,0,NaN,0
3,1,4745,240,1995,Brazilian Grand Prix,brazil,Brazilian,44,26,Olivier Panis,0,0,French,0,27,Ligier,Mugen-Honda,French,26,10,25,0.0,NaN,0,NaN,\N,\N,Not Found,NaN,0,0.00,4,Collision,0,NaN,0
4,1,4744,240,1995,Brazilian Grand Prix,brazil,Brazilian,49,30,Heinz-Harald Frentzen,0,0,German,0,15,Sauber,Ford,Swiss,30,14,24,0.0,NaN,10,NaN,\N,\N,84001,NaN,0,0.00,10,Electrical,0,NaN,0


## Data Preprocessing

#### **New Index Column**

In [26]:
process_df['new_index'] = process_df['Index'] + process_df['driverId']

The above is create a new index column, so that I can track who the model think is going to win races when we get to the prediction stage. 

I then want to make this the new index of my dataframe

In [27]:
process_df.head()

,Index,resultId,raceId,year,race,country,nationality_of_circuit,driverId,number,driver_name,F2_champion,Former_F1_World_Champion,Nationality,home_race,constructorId,constructor,engine_manufacturer,constructor_nationality,number.1,starting_grid_position,positionOrder,points,points_in_previous_race,laps,laps_completed_in_previous_races,time,milliseconds,fastestLap_ms,fastest_lap_from_last_race,number_of_pit_stops,average_time_lost_in_pits,statusId,status,major_regulation_change,laps_in_previous_race,race_win,new_index
0,1,4721,240,1995,Brazilian Grand Prix,brazil,Brazilian,30,1,Michael Schumacher,0,1,German,0,22,Benetton,Renault,Italian,1,2,1,10.0,NaN,71,NaN,01:38:34.154000,5914154,81009,NaN,3,31.83,1,Finished,0,NaN,1,31
1,1,4724,240,1995,Brazilian Grand Prix,brazil,Brazilian,57,8,Mika Häkkinen,0,0,Finnish,0,1,McLaren,Mercedes,British,8,7,4,3.0,NaN,70,NaN,\N,\N,Not Found,NaN,0,0.00,11,+1 Lap,0,NaN,0,58
2,1,4746,240,1995,Brazilian Grand Prix,brazil,Brazilian,94,23,Pierluigi Martini,0,0,Italian,0,18,Minardi,Ford,Italian,23,17,26,0.0,NaN,0,NaN,\N,\N,Not Found,NaN,0,0.00,6,Gearbox,0,NaN,0,95
3,1,4745,240,1995,Brazilian Grand Prix,brazil,Brazilian,44,26,Olivier Panis,0,0,French,0,27,Ligier,Mugen-Honda,French,26,10,25,0.0,NaN,0,NaN,\N,\N,Not Found,NaN,0,0.00,4,Collision,0,NaN,0,45
4,1,4744,240,1995,Brazilian Grand Prix,brazil,Brazilian,49,30,Heinz-Harald Frentzen,0,0,German,0,15,Sauber,Ford,Swiss,30,14,24,0.0,NaN,10,NaN,\N,\N,84001,NaN,0,0.00,10,Electrical,0,NaN,0,50


That looks like it has worked correctly

### Rename Columns

In [28]:
process_df = process_df.rename(columns={'Index': 'race_index'})

In order to enhance clarity within our dataset for predicting F1 race winners, I've updated the column name from 'Index' to 'race_index'

### Drop Redundant Columns

In [29]:
# Define the columns to be dropped
cols_to_drop = ["Index",
    "raceId", "resultId", "country",
    "nationality_of_circuit", "number",
    "fastestLap_ms", "status", "constructor", "positionOrder", "number.1",
    "laps", "major_regulation_change", "time", "milliseconds", "laps_completed_in_previous_races"
]

# Verify that the columns exist in the DataFrame before dropping them
existing_cols_to_drop = [col for col in cols_to_drop if col in process_df.columns]

# Drop the verified columns from the DataFrame
process_df.drop(columns=existing_cols_to_drop, axis=1, inplace=True)

I've removed unnecessary columns from our F1 race dataset, such as raceId, resultId, and others listed, to streamline and focus my predictive modeling efforts, ensuring we're working with the most relevant data for the project.

### Changing Data Types

In [30]:
process_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11235 entries, 0 to 11234
Data columns (total 22 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   race_index                  11235 non-null  int64  
 1   year                        11235 non-null  int64  
 2   race                        11235 non-null  object 
 3   driverId                    11235 non-null  int64  
 4   driver_name                 11235 non-null  object 
 5   F2_champion                 11235 non-null  int64  
 6   Former_F1_World_Champion    11235 non-null  int64  
 7   Nationality                 11235 non-null  object 
 8   home_race                   11235 non-null  int64  
 9   constructorId               11235 non-null  int64  
 10  engine_manufacturer         11235 non-null  object 
 11  constructor_nationality     11235 non-null  object 
 12  starting_grid_position      11235 non-null  int64  
 13  points                      112

The F1 dataset, consisting of 11,235 entries and 21 columns, generally has appropriate data types, with identifiers and categorical variables as integers (`int64`) and strings (`object`), and numeric scores and counters as floating-point numbers (`float64`). However, there are areas needing attention: binary columns like `F2_champion`, `Former_F1_World_Champion`, and `race_win` could be more efficiently stored as boolean (`bool`). The `fastest_lap_from_last_race` column, containing object data types, hints at a mix of strings and numbers, possibly requiring parsing and conversion to a numeric type for any time-related analysis. Moreover, the `laps_completed_in_previous_races` column only contains null values, indicating a decision point—either to populate it with relevant data or remove it if it's not useful. Overall, while the dataset seems robust for predictive modeling, it would benefit from a detailed review to ensure data integrity and optimal memory usage.

#### Turning binary columns into Booleans

In [31]:
# Convert 'F2_champion', 'Former_F1_World_Champion', 'home_race', and 'race_win' columns to boolean type
process_df['F2_champion'] = process_df['F2_champion'].astype(bool)
process_df['Former_F1_World_Champion'] = process_df['Former_F1_World_Champion'].astype(bool)
process_df['home_race'] = process_df['home_race'].astype(bool)
process_df['race_win'] = process_df['race_win'].astype(bool)

# Verify the changes
print(process_df.dtypes)


race_index                      int64
year                            int64
race                           object
driverId                        int64
driver_name                    object
F2_champion                      bool
Former_F1_World_Champion         bool
Nationality                    object
home_race                        bool
constructorId                   int64
engine_manufacturer            object
constructor_nationality        object
starting_grid_position          int64
points                        float64
points_in_previous_race       float64
fastest_lap_from_last_race     object
number_of_pit_stops             int64
average_time_lost_in_pits     float64
statusId                        int64
laps_in_previous_race         float64
race_win                         bool
new_index                       int64
dtype: object


#### Reviewing fastest_lap_from_last_race - not sure why this is an object

In [33]:
print(process_df['fastest_lap_from_last_race'].unique())


[nan '86005' '83003' '86000' '83004' '87008' '84004' '86003' 'Not Found'
 '83008' '87004' '82005' '83000' '83002' '81009' '84001' '83007' '81000'
 '84007' '88005' '89008' '88008' '101007' '96009' '94008' '94006' '95004'
 '95003' '91005' '94001' '96008' '94004' '91003' '92001' '94003' '92009'
 '95001' '94000' '93001' '98005' '97002' '120007' '96001' '90000' '96005'
 '98009' '107001' '93005' '105006' '90001' '90006' '90007' '91009' '91001'
 '106007' '92008' '94005' '93004' '100008' '86002' '88009' '86009' '87005'
 '88002' '87000' '86007' '90009' '85004' '89000' '86004' '85008' '85005'
 '86008' '87003' '92004' '90004' '87007' '88006' '88004' '90005' '87009'
 '90002' '85006' '92002' '98001' '93009' '93000' '96006' '89002' '91008'
 '101009' '92007' '91002' '92005' '91006' '81006' '82008' '80002' '83001'
 '81005' '83009' '81004' '81002' '91004' '95000' '90008' '95006' '98007'
 '93006' '90003' '93002' '96004' '104000' '119008' '116000' '114005'
 '122002' '111007' '113008' '111008' '111006' '1

Looking at this, they should be an integer as they are recorded in Millisecond. I will turn this column into an integret

In [35]:
# Replace 'Not Found' with NaN
process_df['fastest_lap_from_last_race'] = process_df['fastest_lap_from_last_race'].replace('Not Found', np.nan)

# Convert the column to Pandas nullable integer type
process_df['fastest_lap_from_last_race'] = process_df['fastest_lap_from_last_race'].astype('Int64')

### Dropping Duplicates

In [36]:
duplicate_rows = process_df[process_df.duplicated()]
duplicate_rows

,race_index,year,race,driverId,driver_name,F2_champion,Former_F1_World_Champion,Nationality,home_race,constructorId,engine_manufacturer,constructor_nationality,starting_grid_position,points,points_in_previous_race,fastest_lap_from_last_race,number_of_pit_stops,average_time_lost_in_pits,statusId,laps_in_previous_race,race_win,new_index


This snippet shows a step in data preprocessing where we check for duplicate rows in our dataset. The empty DataFrame, `duplicate_rows`, shows (`0 rows x 36 columns`), indicates that there are no duplicate entries in our dataset. This is a positive sign as it means we have a unique set of data points to work with, which is crucial for building a reliable model to predict F1 race winners.

### Feature Engineering / Transformation

#### constructor_points_at_stage_of_season

In [37]:
constructor_points_sum_df = process_df.copy()

In [38]:
# First, ensure 'race_index', 'year', and 'constructorId' are sorted in the order we want to process them
constructor_points_sum_df = constructor_points_sum_df.sort_values(by=['year', 'race_index', 'constructorId'])

# Initialize a new column for corrected constructorId points
constructor_points_sum_df['corrected_constructorId_points'] = 0

# Use a temporary DataFrame to assist with the cumulative sum calculation
temp_df = constructor_points_sum_df.groupby(['year', 'race_index', 'constructorId'])['points'].sum().groupby(level=[0, 2]).cumsum().reset_index()

# Merge this temporary DataFrame back to the original sorted DataFrame
# This step ensures each driver for the constructorId at that race_index sees the summed points on the constructorId level
df_merged = pd.merge(constructor_points_sum_df, temp_df, on=['year', 'race_index', 'constructorId'], how='left')

# The merged DataFrame now has an additional column with the cumulative points which needs to be renamed and checked
df_merged = df_merged.rename(columns={'points_y': 'constructorId_points_at_stage_of_season', 'points_x': 'points'})

# Drop the previously incorrectly calculated column
df_merged.drop(columns=['corrected_constructorId_points'], inplace=True)

In [39]:
# Check the first few rows to ensure the new column has been correctly calculated
df_merged[['race_index', 'driver_name', 'year', 'constructorId', 'points', 'constructorId_points_at_stage_of_season']].head(50)

,race_index,driver_name,year,constructorId,points,constructorId_points_at_stage_of_season
0,1,Mika Häkkinen,1995,1,3.0,4.0
1,1,Mark Blundell,1995,1,1.0,4.0
2,1,Damon Hill,1995,3,0.0,6.0
3,1,David Coulthard,1995,3,6.0,6.0
4,1,Gerhard Berger,1995,6,4.0,6.0
5,1,Jean Alesi,1995,6,2.0,6.0
6,1,Heinz-Harald Frentzen,1995,15,0.0,0.0
7,1,Karl Wendlinger,1995,15,0.0,0.0
8,1,Eddie Irvine,1995,17,0.0,0.0
9,1,Rubens Barrichello,1995,17,0.0,0.0


In [40]:
# Check the first few rows to ensure the new column has been correctly calculated
df_merged[['race_index', 'driver_name', 'year', 'constructorId', 'points', 'constructorId_points_at_stage_of_season']].tail(50)

,race_index,driver_name,year,constructorId,points,constructorId_points_at_stage_of_season
11185,525,Lance Stroll,2023,117,0.0,168.0
11186,525,Fernando Alonso,2023,117,6.0,168.0
11187,525,George Russell,2023,131,10.0,195.0
11188,525,Lewis Hamilton,2023,131,15.0,195.0
11189,525,Kevin Magnussen,2023,210,0.0,8.0
11190,525,Nico Hülkenberg,2023,210,0.0,8.0
11191,525,Nyck de Vries,2023,213,0.0,2.0
11192,525,Yuki Tsunoda,2023,213,0.0,2.0
11193,525,Pierre Gasly,2023,214,0.0,45.0
11194,525,Esteban Ocon,2023,214,0.0,45.0


That seems to have worked - now we have a column that has the cumulative points for the constructor at this stage of the season for each driver.

#### driver_points_at_stage_of_season

In [41]:
driver_points_sum_df = df_merged.copy()

In [42]:
# Use a temporary DataFrame to assist with the cumulative sum calculation for drivers
temp_driver_df = driver_points_sum_df.groupby(['year', 'race_index', 'driver_name'])['points'].sum().groupby(level=[0, 2]).cumsum().reset_index()

# Merge this temporary DataFrame back to the original DataFrame
# This step ensures each driver sees the summed points at that stage of the season
df_merged_with_driver_points = pd.merge(driver_points_sum_df, temp_driver_df, on=['year', 'race_index', 'driver_name'], how='left')

# The merged DataFrame now has an additional column with the cumulative points which needs to be renamed and checked
df_merged_with_driver_points = df_merged_with_driver_points.rename(columns={'points_y': 'driver_points_at_stage_of_season', 'points_x': 'points'})

In [43]:
# Check the first few rows to ensure the new column has been correctly calculated
df_merged_with_driver_points[['race_index', 'year', 'driver_name', 'points', 'driver_points_at_stage_of_season']].head(50)

,race_index,year,driver_name,points,driver_points_at_stage_of_season
0,1,1995,Mika Häkkinen,3.0,3.0
1,1,1995,Mark Blundell,1.0,1.0
2,1,1995,Damon Hill,0.0,0.0
3,1,1995,David Coulthard,6.0,6.0
4,1,1995,Gerhard Berger,4.0,4.0
5,1,1995,Jean Alesi,2.0,2.0
6,1,1995,Heinz-Harald Frentzen,0.0,0.0
7,1,1995,Karl Wendlinger,0.0,0.0
8,1,1995,Eddie Irvine,0.0,0.0
9,1,1995,Rubens Barrichello,0.0,0.0


In [44]:
# Check the first few rows to ensure the new column has been correctly calculated
df_merged_with_driver_points[['race_index', 'year', 'driver_name', 'points', 'driver_points_at_stage_of_season']].tail(50)

,race_index,year,driver_name,points,driver_points_at_stage_of_season
11185,525,2023,Lance Stroll,0.0,38.0
11186,525,2023,Fernando Alonso,6.0,130.0
11187,525,2023,George Russell,10.0,76.0
11188,525,2023,Lewis Hamilton,15.0,119.0
11189,525,2023,Kevin Magnussen,0.0,2.0
11190,525,2023,Nico Hülkenberg,0.0,6.0
11191,525,2023,Nyck de Vries,0.0,0.0
11192,525,2023,Yuki Tsunoda,0.0,2.0
11193,525,2023,Pierre Gasly,0.0,16.0
11194,525,2023,Esteban Ocon,0.0,29.0


That seems to have worked - now we have a column that has the cumulative points for the driver at this stage of the season for each driver. Now we will review the where we are with the dataframe again.

In [48]:
df_merged_with_driver_points.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11235 entries, 0 to 11234
Data columns (total 19 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   race_index                               11235 non-null  int64  
 1   year                                     11235 non-null  int64  
 2   race                                     11235 non-null  object 
 3   driverId                                 11235 non-null  int64  
 4   F2_champion                              11235 non-null  bool   
 5   Former_F1_World_Champion                 11235 non-null  bool   
 6   Nationality                              11235 non-null  object 
 7   home_race                                11235 non-null  bool   
 8   constructorId                            11235 non-null  int64  
 9   engine_manufacturer                      11235 non-null  object 
 10  constructor_nationality                  11235

In [49]:
df_merged_with_driver_points.head()

,race_index,year,race,driverId,F2_champion,Former_F1_World_Champion,Nationality,home_race,constructorId,engine_manufacturer,constructor_nationality,starting_grid_position,points_in_previous_race,fastest_lap_from_last_race,laps_in_previous_race,race_win,new_index,constructorId_points_at_stage_of_season,driver_points_at_stage_of_season
0,1,1995,Brazilian Grand Prix,57,False,False,Finnish,False,1,Mercedes,British,7,NaN,<NA>,NaN,False,58,4.0,3.0
1,1,1995,Brazilian Grand Prix,87,False,False,British,False,1,Mercedes,British,9,NaN,<NA>,NaN,False,88,4.0,1.0
2,1,1995,Brazilian Grand Prix,71,False,True,British,False,3,Renault,British,1,NaN,<NA>,NaN,False,72,6.0,0.0
3,1,1995,Brazilian Grand Prix,14,False,False,British,False,3,Renault,British,3,NaN,<NA>,NaN,False,15,6.0,6.0
4,1,1995,Brazilian Grand Prix,77,False,False,Austrian,False,6,Ferrari,Italian,5,NaN,<NA>,NaN,False,78,6.0,4.0


In [51]:
# Define the columns to be dropped
cols_to_drop = [
    "driver_name", "resultId", "number_of_pit_stops", "average_time_lost_in_pits",
    "statusId", "laps_in_previous_races", "points", "fastest_lap_from_last_race"
]

NB - laps_completed_in_previous_races, I have dropped for the time being. It was full of null values. I will run the first models and use thenm as benchmarks and then see if I can improve the scores. 
The other columns I didn't think were helpful for the following reasons
- `number_of_pit_stops` & `average_time_lost_in_pits` - as we are trying to predict the race winner before the race, this information isn't very helpful at this stage. It may be interesting to see how last year's pit strategy at a particular race will impact next year's race, but it's not so helpful in it's current form.
- `points` - this isn't helpful, as we want to predict forwards and this is captured from `constructorId_points_at_stage_of_season` & `driver_points_at_stage_of_season` columns
- `fastest_lap_from_last_race` and `laps_in_previous_races` are currently missing 

In [52]:

# Verify that the columns exist in the DataFrame before dropping them
existing_cols_to_drop = [col for col in cols_to_drop if col in df_merged_with_driver_points.columns]

# Drop the verified columns from the DataFrame
df_merged_with_driver_points.drop(columns=existing_cols_to_drop, axis=1, inplace=True)

In [53]:
df_merged_with_driver_points.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11235 entries, 0 to 11234
Data columns (total 18 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   race_index                               11235 non-null  int64  
 1   year                                     11235 non-null  int64  
 2   race                                     11235 non-null  object 
 3   driverId                                 11235 non-null  int64  
 4   F2_champion                              11235 non-null  bool   
 5   Former_F1_World_Champion                 11235 non-null  bool   
 6   Nationality                              11235 non-null  object 
 7   home_race                                11235 non-null  bool   
 8   constructorId                            11235 non-null  int64  
 9   engine_manufacturer                      11235 non-null  object 
 10  constructor_nationality                  11235

### One Hot Encoding

In [54]:
# Perform one-hot encoding on the specified columns
df_with_dummies = pd.get_dummies(df_merged_with_driver_points, columns=['race','engine_manufacturer', 'constructor_nationality', 'Nationality'])

# Now df_with_dummies contains the original data along with the one-hot encoded columns


In [58]:
df_with_dummies.head()

,race_index,year,driverId,F2_champion,Former_F1_World_Champion,home_race,constructorId,starting_grid_position,points_in_previous_race,laps_in_previous_race,race_win,new_index,constructorId_points_at_stage_of_season,driver_points_at_stage_of_season,race_70th Anniversary Grand Prix,race_Abu Dhabi Grand Prix,race_Argentine Grand Prix,race_Australian Grand Prix,race_Austrian Grand Prix,race_Azerbaijan Grand Prix,race_Bahrain Grand Prix,race_Belgian Grand Prix,race_Brazilian Grand Prix,race_British Grand Prix,race_Canadian Grand Prix,race_Chinese Grand Prix,race_Dutch Grand Prix,race_Eifel Grand Prix,race_Emilia Romagna Grand Prix,race_European Grand Prix,race_French Grand Prix,race_German Grand Prix,race_Hungarian Grand Prix,race_Indian Grand Prix,race_Italian Grand Prix,race_Japanese Grand Prix,race_Korean Grand Prix,race_Luxembourg Grand Prix,race_Malaysian Grand Prix,race_Mexican Grand Prix,race_Mexico City Grand Prix,race_Miami Grand Prix,race_Monaco Grand Prix,race_Pacific Grand Prix,race_Portuguese Grand Prix,race_Qatar Grand Prix,race_Russian Grand Prix,race_Sakhir Grand Prix,race_San Marino Grand Prix,race_Saudi Arabian Grand Prix,race_Singapore Grand Prix,race_Spanish Grand Prix,race_Styrian Grand Prix,race_SÃ£o Paulo Grand Prix,race_Turkish Grand Prix,race_Tuscan Grand Prix,race_United States Grand Prix,engine_manufacturer_Acer,engine_manufacturer_Arrows,engine_manufacturer_Asiatech,engine_manufacturer_BMW,engine_manufacturer_Cosworth,engine_manufacturer_Ferrari,engine_manufacturer_Ford,engine_manufacturer_Hart,engine_manufacturer_Honda,engine_manufacturer_Mecachrome,engine_manufacturer_Mercedes,engine_manufacturer_Mugen-Honda,engine_manufacturer_Petronas,engine_manufacturer_Peugeot,engine_manufacturer_Playlife,engine_manufacturer_Red Bull,engine_manufacturer_Renault,engine_manufacturer_Supertec,engine_manufacturer_Toro Rosso,engine_manufacturer_Toyota,engine_manufacturer_Yamaha,constructor_nationality_American,constructor_nationality_Austrian,constructor_nationality_British,constructor_nationality_Dutch,constructor_nationality_French,constructor_nationality_German,constructor_nationality_Indian,constructor_nationality_Irish,constructor_nationality_Italian,constructor_nationality_Japanese,constructor_nationality_Malaysian,constructor_nationality_Russian,constructor_nationality_Spanish,constructor_nationality_Swiss,Nationality_American,Nationality_Argentine,Nationality_Australian,Nationality_Austrian,Nationality_Belgian,Nationality_Brazilian,Nationality_British,Nationality_Canadian,Nationality_Chinese,Nationality_Colombian,Nationality_Czech,Nationality_Danish,Nationality_Dutch,Nationality_Finnish,Nationality_French,Nationality_German,Nationality_Hungarian,Nationality_Indian,Nationality_Indonesian,Nationality_Irish,Nationality_Italian,Nationality_Japanese,Nationality_Malaysian,Nationality_Mexican,Nationality_Monegasque,Nationality_New Zealander,Nationality_Polish,Nationality_Portuguese,Nationality_Russian,Nationality_Spanish,Nationality_Swedish,Nationality_Swiss,Nationality_Thai,Nationality_Venezuelan
0,1,1995,57,False,False,False,1,7,NaN,NaN,False,58,4.0,3.0,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,1,1995,87,False,False,False,1,9,NaN,NaN,False,88,4.0,1.0,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,Fals

In our pursuit to predict F1 race winners, it's critical to utilize data that is both relevant and complete. For the inaugural race, indexed as 1 in our dataset, there is no preceding race data available! This absence of prior race information creates gaps in our data (reflected as NaN values), which can mislead our predictive models and skew results. To ensure the integrity and efficacy of our analysis, we remove this first race from our dataset, along with any rows marred by NaNs, thus aligning our data foundation with the solid ground of complete historical context and statistical validity.

In [59]:
# Remove all rows with race_index equal to 1
df_with_dummies = df_with_dummies[df_with_dummies['race_index'] != 1]

In [60]:
df_with_dummies.head()

,race_index,year,driverId,F2_champion,Former_F1_World_Champion,home_race,constructorId,starting_grid_position,points_in_previous_race,laps_in_previous_race,race_win,new_index,constructorId_points_at_stage_of_season,driver_points_at_stage_of_season,race_70th Anniversary Grand Prix,race_Abu Dhabi Grand Prix,race_Argentine Grand Prix,race_Australian Grand Prix,race_Austrian Grand Prix,race_Azerbaijan Grand Prix,race_Bahrain Grand Prix,race_Belgian Grand Prix,race_Brazilian Grand Prix,race_British Grand Prix,race_Canadian Grand Prix,race_Chinese Grand Prix,race_Dutch Grand Prix,race_Eifel Grand Prix,race_Emilia Romagna Grand Prix,race_European Grand Prix,race_French Grand Prix,race_German Grand Prix,race_Hungarian Grand Prix,race_Indian Grand Prix,race_Italian Grand Prix,race_Japanese Grand Prix,race_Korean Grand Prix,race_Luxembourg Grand Prix,race_Malaysian Grand Prix,race_Mexican Grand Prix,race_Mexico City Grand Prix,race_Miami Grand Prix,race_Monaco Grand Prix,race_Pacific Grand Prix,race_Portuguese Grand Prix,race_Qatar Grand Prix,race_Russian Grand Prix,race_Sakhir Grand Prix,race_San Marino Grand Prix,race_Saudi Arabian Grand Prix,race_Singapore Grand Prix,race_Spanish Grand Prix,race_Styrian Grand Prix,race_SÃ£o Paulo Grand Prix,race_Turkish Grand Prix,race_Tuscan Grand Prix,race_United States Grand Prix,engine_manufacturer_Acer,engine_manufacturer_Arrows,engine_manufacturer_Asiatech,engine_manufacturer_BMW,engine_manufacturer_Cosworth,engine_manufacturer_Ferrari,engine_manufacturer_Ford,engine_manufacturer_Hart,engine_manufacturer_Honda,engine_manufacturer_Mecachrome,engine_manufacturer_Mercedes,engine_manufacturer_Mugen-Honda,engine_manufacturer_Petronas,engine_manufacturer_Peugeot,engine_manufacturer_Playlife,engine_manufacturer_Red Bull,engine_manufacturer_Renault,engine_manufacturer_Supertec,engine_manufacturer_Toro Rosso,engine_manufacturer_Toyota,engine_manufacturer_Yamaha,constructor_nationality_American,constructor_nationality_Austrian,constructor_nationality_British,constructor_nationality_Dutch,constructor_nationality_French,constructor_nationality_German,constructor_nationality_Indian,constructor_nationality_Irish,constructor_nationality_Italian,constructor_nationality_Japanese,constructor_nationality_Malaysian,constructor_nationality_Russian,constructor_nationality_Spanish,constructor_nationality_Swiss,Nationality_American,Nationality_Argentine,Nationality_Australian,Nationality_Austrian,Nationality_Belgian,Nationality_Brazilian,Nationality_British,Nationality_Canadian,Nationality_Chinese,Nationality_Colombian,Nationality_Czech,Nationality_Danish,Nationality_Dutch,Nationality_Finnish,Nationality_French,Nationality_German,Nationality_Hungarian,Nationality_Indian,Nationality_Indonesian,Nationality_Irish,Nationality_Italian,Nationality_Japanese,Nationality_Malaysian,Nationality_Mexican,Nationality_Monegasque,Nationality_New Zealander,Nationality_Polish,Nationality_Portuguese,Nationality_Russian,Nationality_Spanish,Nationality_Swedish,Nationality_Swiss,Nationality_Thai,Nationality_Venezuelan
26,2,1995,87,False,False,False,1,17,1.0,70.0,False,89,4.0,1.0,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
27,2,1995,57,False,False,False,1,5,3.0,70.0,False,59,4.0,3.0,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False

Now I will check to see if there are any more NaN values

In [62]:
# Calculate the total number of NaN values in the DataFrame
total_nan_values = df_with_dummies.isna().sum().sum()

# Print the total number of NaN values
print(f"The total number of NaN values in the DataFrame is: {total_nan_values}")


The total number of NaN values in the DataFrame is: 576


In [64]:
# Filter the DataFrame to only include rows with at least one NaN value
rows_with_nan = df_with_dummies[df_with_dummies.isna().any(axis=1)]

# Display those rows
rows_with_nan


,race_index,year,driverId,F2_champion,Former_F1_World_Champion,home_race,constructorId,starting_grid_position,points_in_previous_race,laps_in_previous_race,race_win,new_index,constructorId_points_at_stage_of_season,driver_points_at_stage_of_season,race_70th Anniversary Grand Prix,race_Abu Dhabi Grand Prix,race_Argentine Grand Prix,race_Australian Grand Prix,race_Austrian Grand Prix,race_Azerbaijan Grand Prix,race_Bahrain Grand Prix,race_Belgian Grand Prix,race_Brazilian Grand Prix,race_British Grand Prix,race_Canadian Grand Prix,race_Chinese Grand Prix,race_Dutch Grand Prix,race_Eifel Grand Prix,race_Emilia Romagna Grand Prix,race_European Grand Prix,race_French Grand Prix,race_German Grand Prix,race_Hungarian Grand Prix,race_Indian Grand Prix,race_Italian Grand Prix,race_Japanese Grand Prix,race_Korean Grand Prix,race_Luxembourg Grand Prix,race_Malaysian Grand Prix,race_Mexican Grand Prix,race_Mexico City Grand Prix,race_Miami Grand Prix,race_Monaco Grand Prix,race_Pacific Grand Prix,race_Portuguese Grand Prix,race_Qatar Grand Prix,race_Russian Grand Prix,race_Sakhir Grand Prix,race_San Marino Grand Prix,race_Saudi Arabian Grand Prix,race_Singapore Grand Prix,race_Spanish Grand Prix,race_Styrian Grand Prix,race_SÃ£o Paulo Grand Prix,race_Turkish Grand Prix,race_Tuscan Grand Prix,race_United States Grand Prix,engine_manufacturer_Acer,engine_manufacturer_Arrows,engine_manufacturer_Asiatech,engine_manufacturer_BMW,engine_manufacturer_Cosworth,engine_manufacturer_Ferrari,engine_manufacturer_Ford,engine_manufacturer_Hart,engine_manufacturer_Honda,engine_manufacturer_Mecachrome,engine_manufacturer_Mercedes,engine_manufacturer_Mugen-Honda,engine_manufacturer_Petronas,engine_manufacturer_Peugeot,engine_manufacturer_Playlife,engine_manufacturer_Red Bull,engine_manufacturer_Renault,engine_manufacturer_Supertec,engine_manufacturer_Toro Rosso,engine_manufacturer_Toyota,engine_manufacturer_Yamaha,constructor_nationality_American,constructor_nationality_Austrian,constructor_nationality_British,constructor_nationality_Dutch,constructor_nationality_French,constructor_nationality_German,constructor_nationality_Indian,constructor_nationality_Irish,constructor_nationality_Italian,constructor_nationality_Japanese,constructor_nationality_Malaysian,constructor_nationality_Russian,constructor_nationality_Spanish,constructor_nationality_Swiss,Nationality_American,Nationality_Argentine,Nationality_Australian,Nationality_Austrian,Nationality_Belgian,Nationality_Brazilian,Nationality_British,Nationality_Canadian,Nationality_Chinese,Nationality_Colombian,Nationality_Czech,Nationality_Danish,Nationality_Dutch,Nationality_Finnish,Nationality_French,Nationality_German,Nationality_Hungarian,Nationality_Indian,Nationality_Indonesian,Nationality_Irish,Nationality_Italian,Nationality_Japanese,Nationality_Malaysian,Nationality_Mexican,Nationality_Monegasque,Nationality_New Zealander,Nationality_Polish,Nationality_Portuguese,Nationality_Russian,Nationality_Spanish,Nationality_Swedish,Nationality_Swiss,Nationality_Thai,Nationality_Venezuelan
53,3,1995,95,False,False,False,1,9,NaN,NaN,False,98,6.0,0.0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
95,4,1995,84,False,False,False,27,11,NaN,NaN,False,88,1.0,0.0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False

In [66]:
# Calculate the total number of rows in the DataFrame
total_rows = df_with_dummies.shape[0]

# Calculate the number of rows that contain at least one NaN value
rows_with_nan = df_with_dummies.isna().any(axis=1).sum()

# Calculate the percentage of rows with at least one NaN value
percentage_with_nan = (rows_with_nan / total_rows) * 100

# Print the percentage
print(f"Percentage of rows with at least one NaN value: {percentage_with_nan:.2f}%")


Percentage of rows with at least one NaN value: 2.57%


At this is less than 5%, I will drop these rows.

In [67]:
# Drop all rows that contain any NaN values
df_cleaned = df_with_dummies.dropna()

# Verify changes by displaying the new DataFrame
print(df_cleaned)


       race_index  year  driverId  F2_champion  Former_F1_World_Champion  \
26              2  1995        87        False                     False   
27              2  1995        57        False                     False   
28              2  1995        14        False                     False   
29              2  1995        71        False                      True   
30              2  1995        55        False                     False   
...           ...   ...       ...          ...                       ...   
11230         527  2023       807         True                     False   
11231         527  2023       817        False                     False   
11232         527  2023       852        False                     False   
11233         527  2023       842         True                     False   
11234         527  2023       839        False                     False   

       home_race  constructorId  starting_grid_position  \
26         False            

This looks to have worked

In [68]:
df_cleaned.to_csv('C:/Users/Alex/OneDrive/BrainStation/Data_Science_Bootcamp/Capstone_Project/capstone-Aboard89/model_data.csv', index=False)

## **Conclusion**


In conclusion, the data preparation for the F1 Win Prediction Project involved several crucial preprocessing steps to ensure the data was primed for analysis and modeling:

1. **Creating a New Index**: A new index column was created by adding the 'Index' column and the 'driverId' column to track predictions.

2. **Renaming Columns**: The 'Index' column was renamed to 'race_index'.

3. **Dropping Redundant Columns**: Specific columns that were deemed unnecessary for analysis were dropped from the DataFrame. 

4. **Verifying Data Types**: The types of certain columns were checked to ensure they were correct, although the specific commands for type conversion were commented out and thus not executed.

5. **Dropping Duplicates**: Code to drop duplicate rows was included but commented out, hence not executed.

6. **Handling Data Ranges**: Code to handle unreasonable data ranges was present but commented out.

7. **Feature Engineering - Cumulative Constructor Points**: The DataFrame was sorted, and a new column for constructor points at the stage of the season was calculated using groupby and merge operations.

8. **Feature Engineering - Cumulative Driver Points**: Similarly, a new column for driver points at the stage of the season was calculated using temporary DataFrames to store cumulative sums, which were then merged back into the original DataFrame.

9. **Dropping Verified Columns**: Additional redundant columns were dropped after verifying their existence in the DataFrame.

10. **Exporting Processed Data**: The processed DataFrame was then exported to a CSV file.

11. **One Hot Encoding**: One-hot encoding was performed on specific categorical columns using pd.get_dummies.

12. **Reviewing Column Data Types**: The unique values of the 'fastest_lap_from_last_race' column were printed out, suggesting a review of why it was of the 'object' data type.